In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from PIL import Image
import io
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from sklearn.model_selection import train_test_split
from functools import reduce
from functions import prepare_image, extract_features
import os
from tqdm import tqdm
from scipy.spatial.distance import cosine
np.random.seed(0)

In [45]:
#from flask import Flask, render_template, request, url_for, redirect, session
import pymongo
#import bcrypt
import os,json
import glob
import pandas as pd
from pymongo import MongoClient
from functools import reduce


#app = Flask(__name__)
#app.secret_key = "testing"
client = pymongo.MongoClient("mongodb://127.0.0.1:27017")
db = client.get_database('tag_data')
records = db.register


Collection = db["tags_collection"]

result=[]
filelist = glob.glob('./metadata/*.json') #searching for json files in metadat directory
for filename in filelist:
    print(filename)
    with open(filename) as f:
        file_data = json.load(f)
        result.append(file_data)
        #print(file_data)
  

# Inserting the loaded data in the Collection
# if JSON contains data more than one entry
# insert_many is used else inser_one is used
for file_data in result:
    if isinstance(file_data, list):
        Collection.insert_many(file_data)  
    else:
        Collection.insert_one(file_data)


cursor = Collection.find({},{'_id':False})
  
# Converting cursor to the list 
# of dictionaries
list_cur = list(cursor)
  
# Converting to the JSON
json_data = json.dumps(list_cur, indent = 2) 
#print(json_data)  
# Writing data to file data.json
with open('data.json', 'w') as file:
    file.write(json_data)


hashtag_metadata = []
hashtag_metadata.append(pd.read_json(f'./data.json'))
hashtag_metadata = reduce(lambda x, y: pd.concat([x, y]), hashtag_metadata)
pd.DataFrame.reset_index(hashtag_metadata, drop=True, inplace=True)
hashtag_metadata.tail()



./metadata/fitness.json
./metadata/friends.json
./metadata/selfie.json
./metadata/travel.json
./metadata/nature.json
./metadata/babies.json
./metadata/cars.json
./metadata/food.json
./metadata/architecture.json
./metadata/animals.json
./metadata/wedding.json
./metadata/shopping.json
./metadata/fashion.json


ServerSelectionTimeoutError: 127.0.0.1:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 625f03f134bd82cfe19fc8a3, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('127.0.0.1:27017: [Errno 61] Connection refused',)>]>

In [10]:
json_file_names = os.listdir('metadata')

# Remove the 5 char .json file ending to isolate hashtag name
hashtags = [hashtag[:-5] for hashtag in json_file_names]

# remove '.DS_', '.ipynb_checkp'
non_hashtags = ['.DS_', '.ipynb_checkp']



for non_hashtag in non_hashtags:
    try:
        hashtags.remove(non_hashtag)
    except:
        pass # If we can't remove it, it's already gone
    
    
hashtags

['fitness',
 'friends',
 'selfie',
 'travel',
 'nature',
 'babies',
 'cars',
 'food',
 'architecture',
 'animals',
 'wedding',
 'shopping',
 'fashion']

In [11]:
# convert hashtag metadata into dataframe
hashtag_metadata = []
for hashtag in hashtags: 
    hashtag_metadata.append(pd.read_json(f'metadata/{hashtag}.json'))
hashtag_metadata = reduce(lambda x, y: pd.concat([x, y]), hashtag_metadata)
pd.DataFrame.reset_index(hashtag_metadata, drop=True, inplace=True)
hashtag_metadata.tail()

,post_link,image,search_hashtag,hashtags,image_local_name
1876,https://www.instagram.com/p/CcfHynhJKAz/,https://instagram.fapa1-1.fna.fbcdn.net/v/t51....,fashion,"[#italianfurniture, #renovationmalaysia, #sofa...",3926c05f-a668-496a-9b06-0adb77526c03.jpg
1877,https://www.instagram.com/p/CcfHy3tp-L2/,https://instagram.fapa1-1.fna.fbcdn.net/v/t51....,fashion,"[#workoutmotivation, #o, #powerlifting, #model...",01ef7ce5-9a71-4a90-b9e9-50499df730be.jpg
1878,https://www.instagram.com/p/CcfHxyjqQA2/,https://instagram.fapa1-1.fna.fbcdn.net/v/t51....,fashion,"[#picoftheday, #selfie, #photography, #repost,...",a8dd24fa-d74e-42af-831c-b59bc4b932ec.jpg
1879,https://www.instagram.com/p/CcfHxj-rCru/,https://instagram.fapa1-1.fna.fbcdn.net/v/t51....,fashion,"[#pakistani, #blackhead, #skincare, #whitehead...",fc9bf10e-a445-4a57-ba06-44447d97e267.jpg
1880,https://www.instagram.com/p/CcfHxGeOp4T/,https://instagram.fapa1-2.fna.fbcdn.net/v/t51....,fashion,"[#menwithstyle, #menwithstreetstyleindia, #BU,...",e383160f-386b-4597-b2af-9fa641761f38.jpg


In [12]:
#### store the tag data in redis, so that we can use it as a cache.
def store_tag_data(hashtags):
    from collections import defaultdict

    for hashtag in hashtags:
        df = hashtag_metadata.loc[hashtag_metadata['search_hashtag'] == hashtag]
        hash_tags = defaultdict(int)
        for tags in df['hashtags']:
            for tag in tags:
                hash_tags[tag] += 1

        tag_recommender[hashtag] = sorted(hash_tags.items(), key=lambda item: item[1], reverse=True)[:15]
        
    return tag_recommender

In [12]:
import redis, pickle

redisHost = os.getenv("REDIS_HOST") or "localhost"

db = redis.Redis(host=redisHost, db=1)  

print("Connecting to redis({})".format(redisHost))

def store_tag_redis(db):
    tag_recommender = store_tag_data(hashtags)

    

    ### Push the data into redis so, that we can take hashtags from cache ####

    #### push the username and password and activites into redis cache
    for hashtag in hashtags:
        db.set(hashtag,pickle.dumps(tag_recommender[hashtag]))
    

#store_tag_redis(db)

Connecting to redis(localhost)


In [14]:
### get the recommended tags for a hashtag

def recommend_tags(hashtag):
    details = pickle.loads(db.get(hashtag))

    import re
    recommendation = []   
    for tag in details:
        recommendation.append(tag[0])
    return recommendation

In [13]:
# Remove non-hashtags from hashtag list. 
hashtag_metadata['hashtags'] = hashtag_metadata['hashtags'].apply(
    lambda hashtag_list: [h for h in hashtag_list if h.startswith('#')])


##  For each hashtag we will generate an id and attach that to the image it have. We will have an id for each image.
all_hashtags = [hashtag for hashtags in hashtag_metadata['hashtags'] for hashtag in hashtags]

#set to remove duplicate entries
all_hashtags = sorted(list(set(all_hashtags)))

# Build lookup for finding hashtag number based on hashtag name
hashtag_lookup = {hashtag: i for i, hashtag in enumerate(all_hashtags)}

hashtag_rec_data = []
for i in hashtag_metadata.index:
    hashtag_list = hashtag_metadata.loc[i, 'hashtags']
    for hashtag in hashtag_list:
        hashtag_rec_data.append(
            {'image_id': i,
             'hashtag_id': hashtag_lookup[hashtag],
             'rating': 1}
        )
hashtag_rec_data = pd.DataFrame(hashtag_rec_data)
hashtag_rec_data.tail()



,image_id,hashtag_id,rating
33998,1880,52,1
33999,1880,3724,1
34000,1880,6572,1
34001,1880,42,1
34002,1880,49,1


In [18]:
from pickle import dump
dump(hashtag_lookup, open('hashtag_lookup.pkl', 'wb'))

In [15]:


dump(hashtag_metadata , open('hashtag_metadata.pkl' , 'wb'))

In [17]:
dump(all_hashtags, open('all_hashtags.pk1', 'wb'))

In [38]:
#### We have formatted our data now we will build our neural network on the data ####
img_shape = (160, 160, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

neural_network = tf.keras.Sequential([
  base_model,
  global_average_layer,
])

### for each picture we will extract the features from building our neural network ####
pics = []
for i, row in tqdm(hashtag_metadata.iterrows()):
    name = row['image_local_name']
    hashtag = row['search_hashtag']
    img_path = f'data/{hashtag}/{name}'
    try:
        img = prepare_image(img_path, where='local')
        deep_features = extract_features(img, neural_network)
        pics.append({'pic': img, 
                     'hashtag': hashtag, 
                     'name': name,
                     'deep_features': deep_features})
    except Exception as e:
        error_type = type(e).__name__
        if error_type == "NotFoundError":
            # If a file in the list isn't in 
            # storage, skip it and continue
            pass
        else:
            print(e)




1881it [12:21,  2.54it/s]


In [39]:
pics = pd.DataFrame(pics)
pics.index = pics['name']
pics.head()

,pic,hashtag,name,deep_features
name,,,,
22703b01-e709-42a8-84c9-b8e7363700db.jpg,"(((tf.Tensor(0.99313724, shape=(), dtype=float...",fitness,22703b01-e709-42a8-84c9-b8e7363700db.jpg,"[0.014396438, 0.94950163, 1.4393451, 0.0625216..."
6d22d506-58c9-4769-a21e-e4a642d76a19.jpg,"(((tf.Tensor(0.45882356, shape=(), dtype=float...",fitness,6d22d506-58c9-4769-a21e-e4a642d76a19.jpg,"[0.08373585, 0.16784823, 0.21172367, 0.810663,..."
dcda3a82-3ca3-4b90-8270-87ea24432dbd.jpg,"(((tf.Tensor(-0.23918504, shape=(), dtype=floa...",fitness,dcda3a82-3ca3-4b90-8270-87ea24432dbd.jpg,"[0.031217584, 0.0, 0.50534374, 1.2386708, 0.52..."
139dab5b-6bef-479d-b6b3-e1ba6c196d3f.jpg,"(((tf.Tensor(0.54509807, shape=(), dtype=float...",fitness,139dab5b-6bef-479d-b6b3-e1ba6c196d3f.jpg,"[0.0, 0.0074895667, 0.0, 0.00071224215, 2.1350..."
817e0ebe-c320-45d9-9699-318ad7cbdedf.jpg,"(((tf.Tensor(0.688, shape=(), dtype=float32), ...",fitness,817e0ebe-c320-45d9-9699-318ad7cbdedf.jpg,"[0.0, 0.07952097, 0.1250763, 0.21350689, 0.767..."


In [40]:
##### Build our model , if any image has not more tags we use collaborative filtering to add tags to that  image ####

spark = SparkSession.builder.master('local').getOrCreate()


In [41]:
als = ALS(userCol='image_id',
          itemCol='hashtag_id',
          implicitPrefs=True,
          alpha=40)

als.setSeed(0)

hashtag_spark_df = spark.createDataFrame(hashtag_rec_data)
als_model = als.fit(hashtag_spark_df)
# als_model.write().overwrite().save('als')


Traceback (most recent call last):
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/site-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 102, in dumps
    cp.dump(obj)
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/Users/neerab1652/.pyenv/versions/3.6.7/lib/python3.6/pickle.py", line 476, in save


PicklingError: Could not serialize object: ValueError: Cell is empty